<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = 'hello pytorch. how long can a rnn cell remember?'
chars = 'abcdefghijklmnopqrstuvwxyz ?!.,:;01'
char_list = [i for i in chars] # chars의 값들을 개별로 담음
n_letters = len(char_list) # chars의 길이를 담음

In [ ]:
# string to onehot
def string_to_onehot(string): # string을 받아서 one_hot 처리
  start = np.zeros(shape = len(char_list), dtype = int)
  end = np.zeros(shape = len(char_list), dtype = int)

  start[-2] = 1
  end[-1] = 1
  for i in string:
    idx = char_list.index(i)
    zero = np.zeros(shape = n_letters, dtype = int)

    zero[idx] = 1
    start = np.vstack([start, zero])
  output = np.vstack([start, end])
  return output

In [ ]:
def onehot_to_word(onehot_1): # one_hot처리 된 것을 string으로 반환
  onehot = torch.Tensor.numpy(onehot_1)
  return char_list[onehot.argmax()]

In [ ]:
class RNN(nn.Module): # RNN모델 class 설정
  def __init__(self, input_size, hidden_size, output_size): 
    # input_size: 입력 차원, hidden_size: hidden layer 차원, output_size: output 차원
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.i2h = nn.Linear(input_size, hidden_size)
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.i2o = nn.Linear(hidden_size, output_size)
    self.act_fn = nn.Tanh()

  def forward(self, input, hidden):
    hidden = self.act_fn(self.i2h(input) + self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden

rnn = RNN(n_letters, n_hidden, n_letters)

In [ ]:
loss_func = nn.MSELoss() # loss function으로는 MSE function을 사용
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr) # optimizer로는 torch.optim

In [ ]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor()) # numpy to torch

for i in range(epochs):
  rnn.zero_grad()
  total_loss = 0
  hidden = rnn.init_hidden()